In [1]:
import matplotlib.pyplot as plt
import scipy.constants as sc
import numpy as np
import xtrack as xt
import xcoll as xc
import xobjects as xo
import xpart as xp
from pathlib import Path
import lossmaps as lm
import cernlayoutdb as layout

 If you are on SWAN and did not expect this warning, then restart and load the 'NXCals Pro' software stack.


In [10]:
#No idea what it is doing
import cx_Oracle
cx_Oracle.init_oracle_client(lib_dir="/Users/gebruiker/packages/instantclient_19_8")
## cx_Oracle.init_oracle_client(lib_dir="/Users/your_username/Downloads/instantclient_19_8")

cx_Oracle.clientversion()  
print(cx_Oracle.version)
cx_Oracle.clientversion()

8.3.0


(19, 8, 0, 0, 0)

BeamElement.name_associated_aperture in thick line

In [3]:
path_lines = Path('/Users/gebruiker/sps_simulations/injection_lines')
line = xt.Line.from_json(Path(path_lines,'injection_thick_no_ap_with_deferred_expressions.json'))

Loading line from dict:   0%|          | 0/4426 [00:00<?, ?it/s]

Done loading line from dict.           


In [32]:
#Set cavitiy voltages correctly
line['acl.31735'].frequency = 800e6
line['acl.31735'].voltage = 2 * 0.18e6
line['acl.31735'].lag = 180

line['actcse.31632'].frequency = 200e6
line['actcse.31632'].voltage = 4.5e6
line['actcse.31632'].lag = 180

In [11]:
db = layout.DB()
aperture_info_list = db.get_apertures(machine='SPS', version='EYETS 2024-2025')

l_uniques = {'Bend' : np.unique([name[:4] for name in line.get_elements_of_type(xt.Bend)[1]]),
             'Quadrupole': np.unique([name[:4] for name in line.get_elements_of_type(xt.Quadrupole)[1]]),
             'Sextupole': np.unique([name[:4] for name in line.get_elements_of_type(xt.Sextupole)[1]]),
             'Octupole': np.unique([name[:4] for name in line.get_elements_of_type(xt.Octupole)[1]]),
             'Multipole': np.unique([name[:4] for name in line.get_elements_of_type(xt.Multipole)[1]]),
             'Cavity': np.unique([name[:4] for name in line.get_elements_of_type(xt.Cavity)[1]])}

In [12]:
dico_types = {}
for elem in aperture_info_list:
    name = elem[1].lower()
    if any(name.startswith(pattern) for patterns in l_uniques.values() for pattern in patterns):
        if name[:4] not in dico_types:
            dico_types[name[:4]] = [elem[2]]
        else:
            if elem[2] not in dico_types[name[:4]]:
                dico_types[name[:4]].append(elem[2])

In [13]:
aperture_data_per_unique = {}

for key in dico_types:
    if len(dico_types[key])== 1:
        # print(key)
        # print(np.unique([elem[4][1][0] for elem in aperture_info_list if elem[2]==dico_types[key][0]]))
        # print(np.unique([elem[4][1][1] for elem in aperture_info_list if elem[2]==dico_types[key][0]]))
        # print(np.unique([elem[4][1][2] for elem in aperture_info_list if elem[2]==dico_types[key][0]]))
        # print(np.unique([elem[4][1][3] for elem in aperture_info_list if elem[2]==dico_types[key][0]]))
        # print(np.unique([elem[4][0] for elem in aperture_info_list if elem[2]==dico_types[key][0]]))
        # print(' ')
        aperture_data_per_unique[key] = {'aper_type' : str(np.unique([elem[4][0] for elem in aperture_info_list if elem[2]==dico_types[key][0]])[0]),
                                       'max_x' : float(np.unique([elem[4][1][0] for elem in aperture_info_list if elem[2]==dico_types[key][0]])[0]),
                                       'max_y' : float(np.unique([elem[4][1][1] for elem in aperture_info_list if elem[2]==dico_types[key][0]])[0]),
                                       'a' : float(np.unique([elem[4][1][2] for elem in aperture_info_list if elem[2]==dico_types[key][0]])[0]),
                                       'b' : float(np.unique([elem[4][1][3] for elem in aperture_info_list if elem[2]==dico_types[key][0]])[0])}

In [14]:
aperture_data_per_unique['actc'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : 0.065,
                                       'max_y' : 0.065,
                                       'a' : 0.065,
                                       'b' : 0.065}

aperture_data_per_unique['lqsa'] = {'aper_type' : 'RECTANGLE',
                                       'max_x' : 0.08,
                                       'max_y' : 0.08,
                                       'a' : None,
                                       'b' : None,
                                       'Note' : 'Simplified to a rectangle'}

aperture_data_per_unique['qe.'] = {'aper_type' : 'RECTANGLE',
                                       'max_x' : 0.0661,
                                       'max_y' : 0.0661,
                                       'a' : None,
                                       'b' : None,
                                       'Note' : 'Simplified and approximated'}

aperture_data_per_unique['qecd'] = {'aper_type' : 'RECTANGLE',
                                       'max_x' : 0.0661,
                                       'max_y' : 0.0661,
                                       'a' : None,
                                       'b' : None,
                                       'Note' : 'Simplified and approximated'}

aperture_data_per_unique['qfa.'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.05378,
                                       'b' : 0.05378,
                                       'Note' : 'Simplified and approximated'}

aperture_data_per_unique['lod.'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.0415,
                                       'b' : 0.0415,
                                       'Note' : 'From old file, no CAD in LDB'}

aperture_data_per_unique['adkcv'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.1,
                                       'b' : 0.1,
                                       'Note' : 'no CAD in LDB'}

aperture_data_per_unique['adksv'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.1,
                                       'b' : 0.1,
                                       'Note' : 'no CAD in LDB'}

aperture_data_per_unique['acfca'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.1,
                                       'b' : 0.1,
                                       'Note' : 'no CAD in LDB'}

aperture_data_per_unique['mdhb'] = {'aper_type' : 'RECTANGLE',
                                       'max_x' : 0.36,
                                       'max_y' : 0.118,
                                       'a' : None,
                                       'b' : None}

aperture_data_per_unique['mdhd'] = {'aper_type' : 'RECTANGLE',
                                       'max_x' : 0.32,
                                       'max_y' : 0.09,
                                       'a' : None,
                                       'b' : None}

aperture_data_per_unique['mdhw'] = {'aper_type' : 'RECTANGLE',
                                       'max_x' : 0.2,
                                       'max_y' : 0.2,
                                       'a' : None,
                                       'b' : None}

aperture_data_per_unique['mdvw'] = {'aper_type' : 'RECTANGLE',
                                       'max_x' : 0.2,
                                       'max_y' : 0.2,
                                       'a' : None,
                                       'b' : None}

aperture_data_per_unique['mkdh'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.103,
                                       'b' : 0.103,
                                       'Note' : 'mkdhb, no mkdha and even b is weird drawing'}

aperture_data_per_unique['mkdv'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.103,
                                       'b' : 0.103}

aperture_data_per_unique['mke.'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.103,
                                       'b' : 0.103}

aperture_data_per_unique['mkp'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.103,
                                       'b' : 0.103}

aperture_data_per_unique['mkqv'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.08102,
                                       'b' : 0.08102}

aperture_data_per_unique['mplh'] = {'aper_type' : 'RECTANGLE',
                                       'max_x' : 0.05,
                                       'max_y' : 0.38617,
                                       'a' : None,
                                       'b' : None}

aperture_data_per_unique['mse.'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.1339,
                                       'b' : 0.1339}

aperture_data_per_unique['mst.'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.1339,
                                       'b' : 0.1339,
                                       'Note' : 'No cad but its supposed to be one tank'}

aperture_data_per_unique['zs.'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.10755,
                                       'b' : 0.10755}


aperture_data_per_unique['acfca'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.1,
                                       'b' : 0.1}

In [15]:
for key in aperture_data_per_unique:
    aper_type = aperture_data_per_unique[key]['aper_type']
    max_x = aperture_data_per_unique[key]['max_x']
    max_y = aperture_data_per_unique[key]['max_y']
    a = aperture_data_per_unique[key]['a']
    b = aperture_data_per_unique[key]['b']
    
    if aper_type == 'CIRCLE':
        aperture_data_per_unique[key]['xt_elem'] = xt.LimitEllipse(a=a, b=b)
    elif aper_type == 'ELLIPSE':
        aperture_data_per_unique[key]['xt_elem'] = xt.LimitEllipse(a=a, b=b)
    elif aper_type == 'RECTANGLE':
        aperture_data_per_unique[key]['xt_elem'] = xt.LimitRect(min_x=-max_x, max_x=max_x, min_y=-max_y, max_y=max_y)
    elif aper_type == 'RECTELLIPSE':
        aperture_data_per_unique[key]['xt_elem'] = xt.LimitRectEllipse(max_x=max_x, max_y=max_y, a=a, b=b)
    elif aper_type == 'RACETRACK':
        aperture_data_per_unique[key]['xt_elem'] = xt.LimitRacetrack(min_x=-max_x, max_x=max_x, min_y=-max_y, max_y=max_y, a=a, b=b)

In [38]:
# Update insert_names to handle variable-length keys
insert_names = [
    {f'{name}_aper_upstream': index, f'{name}_aper_downstream': index + 1}
    for index, name in enumerate(line.element_names)
    if any(name.lower().startswith(key) for key in aperture_data_per_unique)
]
insert_names = {kk: vv for dct in insert_names for kk, vv in dct.items()}
idxs = list(insert_names.values())
names = list(insert_names.keys())
max_length = max(max(map(len, line.element_names)), max(map(len, names)))
element_names = np.array(line.element_names, dtype=f'<U{max_length}')
names = np.array(names, dtype=f'<U{max_length}')
element_names = np.insert(element_names, idxs, names)

# Update insert_elements to handle variable-length keys
insert_elements = [
    {
        f'{name}_aper_upstream': aperture_data_per_unique[next(key for key in aperture_data_per_unique if name.lower().startswith(key))]['xt_elem'].copy(),
        f'{name}_aper_downstream': aperture_data_per_unique[next(key for key in aperture_data_per_unique if name.lower().startswith(key))]['xt_elem'].copy()
    }
    for name in line.element_names
    if any(name.lower().startswith(key) for key in aperture_data_per_unique)
]
insert_elements = {kk: vv for dct in insert_elements for kk, vv in dct.items()}

# Update line's element names and element dictionary
line.element_names = element_names.tolist()
line.element_dict = {**line.element_dict, **insert_elements}

In [40]:
slice_factor = 1 
slicer = xt.slicing.Slicer(line,
                   slicing_strategies=[xt.Strategy(slicing=xt.Teapot(1)),
                                              xt.Strategy(slicing=xt.Teapot(2), element_type=xt.Bend),
                                              xt.Strategy(slicing=xt.Teapot(2*slice_factor), element_type=xt.Quadrupole),
                                              xt.Strategy(slicing=xt.Teapot(32*slice_factor), name='mqxa.*'),
                                              xt.Strategy(slicing=xt.Teapot(32*slice_factor), name='mqxb.*'),
                                              xt.Strategy(slicing=xt.Teapot(4), name='mqxb\\..*'),
                                              xt.Strategy(slicing=xt.Teapot(4), name='mbrb\\..*'),
                                              xt.Strategy(slicing=xt.Teapot(4), name='mbrc\\..*'),
                                              xt.Strategy(slicing=xt.Teapot(4), name='mbrs\\..*'),
                                              xt.Strategy(slicing=xt.Teapot(4), name='mqwa\\..*'),
                                              xt.Strategy(slicing=xt.Teapot(4), name='mqwb\\..*'),
                                              xt.Strategy(slicing=xt.Teapot(4*slice_factor), name='mqy\\..*'),
                                              xt.Strategy(slicing=xt.Teapot(4*slice_factor), name='mqm\\..*'),
                                              xt.Strategy(slicing=xt.Teapot(4*slice_factor), name='mqmc\\..*'),
                                              xt.Strategy(slicing=xt.Teapot(4*slice_factor), name='mqml\\..*'),
                                              xt.Strategy(slicing=xt.Teapot(2*slice_factor), name='mqtlh\\..*'),
                                              xt.Strategy(slicing=xt.Teapot(2*slice_factor), name='mqtli\\..*'),
                                              xt.Strategy(slicing=xt.Teapot(2*slice_factor), name='mqt\\..*'),
                                              xt.Strategy(slicing=None, element_type=xt.Solenoid),
                                              xt.Strategy(slicing=None, name='qe\\..*'), 
                                              xt.Strategy(slicing=None, name='qfa\\..*'),
                                              xt.Strategy(slicing=None, name='qecd\\..*')])

# Test class

In [8]:
import apertls

In [4]:
# line = xt.Line.from_json(Path(path_lines,'injection_thick_no_ap_with_deferred_expressions.json'))
line = xt.Line.from_json(Path(path_lines,'federica_line_no_ap.json'))

Loading line from dict:   0%|          | 0/4482 [00:00<?, ?it/s]

Done loading line from dict.           


In [5]:
#Set cavitiy voltages correctly
line['acl.31735'].frequency = 800e6
line['acl.31735'].voltage = 2 * 0.18e6
line['acl.31735'].lag = 180

line['actcse.31632'].frequency = 200e6
line['actcse.31632'].voltage = 4.5e6
line['actcse.31632'].lag = 180

In [6]:
line.slice_thick_elements(slicing_strategies=[xt.Strategy(slicing=xt.Teapot(4))])

Slicing line:   0%|          | 0/4482 [00:00<?, ?it/s]

{'sps$start': ['sps$start'],
 'begi.10010': ['begi.10010'],
 'qf.10010': ['qf.10010_entry',
  'qf.10010..entry_map',
  'drift_qf.10010..0',
  'qf.10010..0',
  'drift_qf.10010..1',
  'qf.10010..1',
  'drift_qf.10010..2',
  'qf.10010..2',
  'drift_qf.10010..3',
  'qf.10010..3',
  'drift_qf.10010..4',
  'qf.10010..exit_map',
  'qf.10010_exit'],
 'drift_0': ['drift_0'],
 'mba.10030': ['mba.10030_entry',
  'mba.10030..entry_map',
  'drift_mba.10030..0',
  'mba.10030..0',
  'drift_mba.10030..1',
  'mba.10030..1',
  'drift_mba.10030..2',
  'mba.10030..2',
  'drift_mba.10030..3',
  'mba.10030..3',
  'drift_mba.10030..4',
  'mba.10030..exit_map',
  'mba.10030_exit'],
 'drift_1': ['drift_1'],
 'mba.10050': ['mba.10050_entry',
  'mba.10050..entry_map',
  'drift_mba.10050..0',
  'mba.10050..0',
  'drift_mba.10050..1',
  'mba.10050..1',
  'drift_mba.10050..2',
  'mba.10050..2',
  'drift_mba.10050..3',
  'mba.10050..3',
  'drift_mba.10050..4',
  'mba.10050..exit_map',
  'mba.10050_exit'],
 'drift_2'

In [16]:
ap_manager = apertls.ManageApertures(line, machine='SPS', version='EYETS 2024-2025')

In [17]:
ap_manager.associated_apertures_naming(complementary_apertures=aperture_data_per_unique)

No matching aperture for qf.10010..entry_map in complementary dictionary
No matching aperture for qf.10010..0 in complementary dictionary
No matching aperture for qf.10010..1 in complementary dictionary
No matching aperture for qf.10010..2 in complementary dictionary
No matching aperture for qf.10010..3 in complementary dictionary
No matching aperture for qf.10010..exit_map in complementary dictionary
No matching aperture for mba.10030..entry_map in complementary dictionary
No matching aperture for mba.10030..0 in complementary dictionary
No matching aperture for mba.10030..1 in complementary dictionary
No matching aperture for mba.10030..2 in complementary dictionary
No matching aperture for mba.10030..3 in complementary dictionary
No matching aperture for mba.10030..exit_map in complementary dictionary
No matching aperture for mba.10050..entry_map in complementary dictionary
No matching aperture for mba.10050..0 in complementary dictionary
No matching aperture for mba.10050..1 in com

In [18]:
ap_manager.install_apertures()

In [19]:
line.check_aperture()

Checking aperture:   0%|          | 0/19665 [00:00<?, ?it/s]

Done checking aperture.           
6660 thin elements miss associated aperture (upstream):
['qd.10110..entry_map',
 'qd.10110..0',
 'qd.10110..1',
 'qd.10110..2',
 'qd.10110..3',
 'qd.10110..exit_map',
 'mbb.10130..entry_map',
 'mbb.10130..0',
 'mbb.10130..1',
 'mbb.10130..2',
 'mbb.10130..3',
 'mbb.10130..exit_map',
 'mbb.10150..entry_map',
 'mbb.10150..0',
 'mbb.10150..1',
 'mbb.10150..2',
 'mbb.10150..3',
 'mbb.10150..exit_map',
 'mba.10170..entry_map',
 'mba.10170..0',
 'mba.10170..1',
 'mba.10170..2',
 'mba.10170..3',
 'mba.10170..exit_map',
 'mba.10190..entry_map',
 'mba.10190..0',
 'mba.10190..1',
 'mba.10190..2',
 'mba.10190..3',
 'mba.10190..exit_map',
 'lsf.10205..entry_map',
 'lsf.10205..0',
 'lsf.10205..1',
 'lsf.10205..2',
 'lsf.10205..3',
 'lsf.10205..exit_map',
 'qf.10210..entry_map',
 'qf.10210..0',
 'qf.10210..1',
 'qf.10210..2',
 'qf.10210..3',
 'qf.10210..exit_map',
 'mba.10230..entry_map',
 'mba.10230..0',
 'mba.10230..1',
 'mba.10230..2',
 'mba.10230..3',
 'mba.102

,s,element_type,name,isthick,isreplica,parent_name,iscollective,element,s_start,s_center,s_end,is_aperture,i_aperture_upstream,s_aperture_upstream,i_aperture_downstream,s_aperture_downstream,misses_aperture_upstream,misses_aperture_downstream,has_aperture_problem
0,0.000000,Marker,sps$start,False,False,None,False,Marker(),0.000000,0.000000,0.000000,False,NaN,NaN,NaN,NaN,False,False,False
1,0.000000,Marker,begi.10010,False,False,None,False,Marker(),0.000000,0.000000,0.000000,False,NaN,NaN,NaN,NaN,False,False,False
2,0.000000,Marker,qf.10010_entry,False,False,None,False,Marker(),0.000000,0.000000,0.000000,False,NaN,NaN,NaN,NaN,False,False,False
3,0.000000,ThinSliceQuadrupoleEntry,qf.10010..entry_map,False,False,qf.10010,False,ThinSliceQuadrupoleEntry(parent_name='qf.10010...,0.000000,0.000000,0.000000,False,NaN,NaN,NaN,NaN,False,False,False
4,0.000000,DriftSliceQuadrupole,drift_qf.10010..0,True,False,qf.10010,False,"DriftSliceQuadrupole(parent_name='qf.10010', _...",0.000000,0.154250,0.308500,False,NaN,NaN,NaN,NaN,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19750,6911.148189,Drift,bph.63608,True,False,None,False,Drift(length=0.275),6911.148189,6911.285689,6911.423189,False,NaN,NaN,NaN,NaN,False,False,False
19751,6911.423189,Drift,drift_1907,True,False,None,False,Drift(length=0.095),6911.423189,6911.470689,6911.518189,False,NaN,NaN,NaN,NaN,False,False,False
19752,6911.518189,Marker,end.10010,False,False,None,False,Marker(),6911.518189,6911.518189,6911.518189,False,NaN,NaN,NaN,NaN,False,False,False
19753,6911.518189,Marker,sps$end,False,False,None,False,Marker(),6911.518189,6911.518189,6911.518189,False,NaN,NaN,NaN,NaN,False,False,False


In [20]:
line.twiss()

TwissTable: 19755 rows, 62 cols
name                            s             x            px             y            py ...
sps$start                       0             0             0             0             0
begi.10010                      0             0             0             0             0
qf.10010_entry                  0             0             0             0             0
qf.10010..entry_map             0             0             0             0             0
drift_qf.10010..0               0             0             0             0             0
qf.10010..0                0.3085             0             0             0             0
drift_qf.10010..1          0.3085             0             0             0             0
qf.10010..1               1.13117             0             0             0             0
drift_qf.10010..2         1.13117             0             0             0             0
qf.10010..2               1.95383             0             0   

In [16]:
dic_test = ap_manager.find_associated_apertures()

In [21]:
line.to_json(Path(path_lines, 'federica_line_with_ap.json'))